In [45]:
import torch
import PIL
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [5]:
model = torch.hub.load("pytorch/vision", "resnet50", weights="IMAGENET1K_V2")

Downloading: "https://github.com/pytorch/vision/zipball/main" to C:\Users\Mike/.cache\torch\hub\main.zip


In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
for p in model.parameters():
    p.requires_grad = False

In [21]:
# class Sim(torch.nn.Module):
#     def __init__(self) -> None:
#         super().__init__(Sim, self).__init__()

#     def forward(self, x):
#         return x

In [ ]:
# model.avgpool = Sim()

In [33]:
model.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 1024),
    torch.nn.Tanh(),
    torch.nn.Linear(1024, 256),
    torch.nn.Tanh(),
    torch.nn.Linear(256, 64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 4),
    torch.nn.Tanh(),
    torch.nn.Linear(4, 1),
    torch.nn.ReLU()
)

In [34]:
model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
data = np.load('datasets/images.npy/images.npy')

In [40]:
labels = pd.read_csv('datasets/labels.csv')['count']

In [43]:
data.shape

(2000, 480, 640, 3)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data, labels.to_numpy(), test_size=0.15,)

In [49]:
train_ds = TensorDataset(
    torch.from_numpy(X_train).type(torch.float32), 
    torch.from_numpy(y_train).type(torch.float32)
)
train_dl = DataLoader(train_ds, batch_size=512, shuffle=True)

In [50]:
test_ds = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_dl = DataLoader(test_ds, batch_size=512, shuffle=True)

In [52]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 50
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        outputs = model(x_b.to('cuda'))
        loss_value = loss(outputs, y_b.to('cuda'))
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')